In [1]:
import wbgapi as wb   # I use wb as a namespace in all my work
import os
print("PYTHONPATH =", os.environ.get("PYTHONPATH"))
from internal.models import *
from internal.dags.extract import wbapi_extract
from internal.dags.transform import wbapi_transform

PYTHONPATH = C:/Users/Admin/Downloads/Project/Github/ETL_Project
PYTHONPATH = C:/Users/Admin/Downloads/Project/Github/ETL_Project


In [2]:
extract_obj = wbapi_extract()
transform_obj = wbapi_transform()



In [3]:
# obj_test = object.series.get_data(SeriesDataInput(id='all'))
# print(obj_test)
# obj_test.to_csv(path = "")

In [5]:
# for db in range(1, 16):
from pyspark.sql import SparkSession

spark = SparkSession.builder
obj_extract = extract_obj.economy.dataframe_display(EconomyDataFrameInput(id='all'))
obj_dataFrame_transform = transform_obj.economy.transform_economy_dataframe(spark = spark, df = obj_extract)
print(obj_extract["Aggregate"])
print(obj_dataFrame_transform)
# print(type(obj_extract))
print(type(obj_dataFrame_transform))
# df = obj_extract.to_frame()
# print(df.index)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  # để không cắt ngắn văn bản dài
pd.set_option('display.expand_frame_repr', False)  # tránh wrap dòng



0      False
1       True
2      False
3       True
4      False
       ...  
261    False
262    False
263    False
264    False
265    False
Name: Aggregate, Length: 266, dtype: bool
                    Country_Name Longitude Latitude Region Admin_Region  \
0                          Aruba  -70.0167  12.5167    LCN                
1    Africa Eastern and Southern                                          
2                    Afghanistan   69.1761  34.5228    MEA          MNA   
3     Africa Western and Central                                          
4                         Angola    13.242 -8.81155    SSF          SSA   
..                           ...       ...      ...    ...          ...   
261                       Kosovo    20.926   42.565    ECS          ECA   
262                  Yemen, Rep.   44.2075   15.352    MEA          MNA   
263                 South Africa   28.1871  -25.746    SSF          SSA   
264                       Zambia   28.2937 -15.3982    SSF       

In [ ]:
print(type(obj_dataFrame_transform))
# df =pd.DataFrame(obj_extract.items)


In [ ]:
display(obj_dataFrame_transform)

In [ ]:
# print(obj_extract.items)
# df = pd.DataFrame(obj_extract.items)
# df